# Exercises
- The end result of this exercise should be a Jupyter notebook named model.

- Using saas.csv or log data from API usage or store_item_sales


In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

import warnings
warnings.filterwarnings("ignore")


##  1. Split data (train/validate/test) and resample by any period, except daily, and aggregate using the sum.


In [2]:
#acquiring csv
saas_df= pd.read_csv('saas.csv')

In [3]:
saas_df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [4]:
#Looking at value counts in amount to decide whether to change to int or not
saas_df.Amount.value_counts()

0.0     496410
5.0     178212
10.0     70706
15.0     26474
20.0     18656
Name: Amount, dtype: int64

In [5]:
#Looking at datatypes
saas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Month_Invoiced     790458 non-null  object 
 1   Customer_Id        790458 non-null  int64  
 2   Invoice_Id         790458 non-null  int64  
 3   Subscription_Type  790458 non-null  float64
 4   Amount             790458 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 30.2+ MB


In [6]:
# Setting datatime datatype
saas_df.Month_Invoiced= pd.to_datetime(saas_df.Month_Invoiced)

In [7]:
# Set the index to be the datetime variable.
saas_df= saas_df.set_index('Month_Invoiced').sort_index()

In [8]:
#Changing Subscription Type column to integer datatype
saas_df.Subscription_Type= saas_df.Subscription_Type.astype('int')

In [9]:
#Changing amount column to integer datatype
saas_df.Amount= saas_df.Amount.astype('int')

In [10]:
#Verify datatype was changed
saas_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 790458 entries, 2014-01-31 to 2017-12-31
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   Customer_Id        790458 non-null  int64
 1   Invoice_Id         790458 non-null  int64
 2   Subscription_Type  790458 non-null  int64
 3   Amount             790458 non-null  int64
dtypes: int64(4)
memory usage: 30.2 MB


In [23]:
# aggregate or resample daily by summing dollars & items sold
df_resampled = saas_df.resample('d')['Amount'].sum()

In [24]:
#Looking at the first 5 values of df_resampled
df_resampled.head()

Month_Invoiced
2014-01-31    35850
2014-02-01        0
2014-02-02        0
2014-02-03        0
2014-02-04        0
Freq: D, Name: Amount, dtype: int64

In [26]:
# set train size to be 50% of total 
train_size = int(len(saas_df) * 0.5)
train_size

395229

In [27]:
# set validate size to be 30% of total 
validate_size = int(len(saas_df) * 0.3)
validate_size

237137

In [29]:
# set test size to be number of rows remaining. 
test_size = int(len(saas_df) - train_size - validate_size)
test_size

158092

## 2. Forecast, plot and evaluate using each of the 4 parametric based methods we discussed:
    - Simple Average
    - Moving Average
    - Holt's Linear Trend Model
    - Based on previous year/month/etc., this is up to you.